In [3]:
from Bio import SeqIO
import numpy as np
from numpy import inf

In [4]:
def init_A(m,n):
    a=np.full((m,n),-inf)
    a[0][0]=0
    return a

def init_S(m,n,b):
    a=np.full((m,n),-inf)
    a[0][0]=0
    a[1][0]=b
    return a

def init_T(m,n,b):
    a=np.full((m,n),-inf)
    a[0][0]=0
    a[0][1]=b
    return a

def init_OPT(m,n,b):
    a=np.full((m,n),-inf)
    a[0][0]=0
    a[0][1]=b
    a[1][0]=b
    return a
def init_back_ptrs(m, n):
    tables = ('M', 'S_gap', 'T_gap')
    back_ptrs = {table: np.full((m,n),-inf) for table in tables}
    return back_ptrs  

In [5]:
init_back_ptrs(2,3)

{'M': array([[-inf, -inf, -inf],
        [-inf, -inf, -inf]]),
 'S_gap': array([[-inf, -inf, -inf],
        [-inf, -inf, -inf]]),
 'T_gap': array([[-inf, -inf, -inf],
        [-inf, -inf, -inf]])}

In [6]:
def match_bases(s,t,match,mismatch):
    if s==t:
        return match
    else:
        return mismatch

In [7]:
def filling_A(sj,ti,match,mismatch,A,S,T,back_ptrs,i,j):
    A[i][j]=match_bases(sj,ti,match,mismatch)+max(A[i-1][j-1],S[i-1][j-1],T[i-1][j-1])
    max_val=max(A[i-1][j-1],S[i-1][j-1],T[i-1][j-1])
    if max_val==A[i-1][j-1]:
        back_ptrs['M'][i][j]=0
    elif max_val==S[i-1][j-1]:
        back_ptrs['M'][i][j]=1
    elif max_val==T[i-1][j-1]:
        back_ptrs['M'][i][j]=2
    return A,back_ptrs
def filling_S(sj,ti,match,mismatch,A,S,T,back_ptrs,i,j):
    S[i][j]=b+max(A[i][j-1],T[i][j-1])
    max_val=max(A[i][j-1],T[i][j-1])
    if max_val==A[i][j-1]:
        back_ptrs['S_gap'][i][j]=0
    
    elif max_val==T[i][j-1]:
        back_ptrs['S_gap'][i][j]=2
    return S,back_ptrs
def filling_T(sj,ti,match,mismatch,A,S,T,back_ptrs,i,j):
    T[i][j]=b+max(A[i-1][j],S[i-1][j])
    max_val=max(A[i-1][j],S[i-1][j])
    if max_val==A[i-1][j]:
        back_ptrs['T_gap'][i][j]=0
    
    elif max_val==S[i-1][j]:
        back_ptrs['T_gap'][i][j]=1
    return T,back_ptrs

        

In [8]:
def filling_matrices(b,s,t,match,mismatch):
    
    m=len(s)+1
    n=len(t)+1
    A=init_A(m,n)
    S=init_S(m,n,b)
    T=init_T(m,n,b)
    ptrs=init_back_ptrs(m, n)
    
    for i in range(1, m):
        for j in range(1, n):
            #print(i,j)
            tj=t[j-1]
            si=s[i-1]
            #print(match_bases(sj,ti,match,mismatch))
            A,ptrs=filling_A(si,tj,match,mismatch,A,S,T,ptrs,i,j)
            S,ptrs=filling_S(si,tj,match,mismatch,A,S,T,ptrs,i,j)
            T,ptrs=filling_T(si,tj,match,mismatch,A,S,T,ptrs,i,j)
    return A,S,T,ptrs
            

In [9]:
def back_trace(ptr,start_table,i,j,s,t):
    seqs=''
    seqt=''
    table=['M','S_gap','T_gap']
    while( i>=0 or j>=0):
        if start_table=='M':
            seqs+=s[i]
            seqt+=t[j]
            #print('M',seqs,seqt,i,j,ptr[start_table][i+1][j+1])
            
            if i>=0 and j>=0:
                start_table=table[int(ptr[start_table][i+1][j+1])]
                i-=1;j-=1
            else:
                break
            
            
        elif start_table=='S_gap':
            seqs+='-'
            seqt+=t[j]
            #print('S',seqs,seqt,i,j,ptr[start_table][i+1][j+1])

        
            if j>=0:
                start_table=table[int(ptr[start_table][i+1][j+1])]
                j-=1
            else:
                
                break
            
        elif start_table=='T_gap':
            seqs+=s[i]
            seqt+='-'
            #print('T',seqs,seqt,i,j,ptr[start_table][i+1][j+1])

            if i>=0:
                start_table=table[int(ptr[start_table][i+1][j+1])]
                i-=1
            else:
                
                break
           

    print(seqs,seqt,i,j)
    return seqs[::-1],seqt[::-1]
            

In [10]:
def load_fasta(filename):

    fasta_seqs = SeqIO.parse(open(filename), 'fasta')
    name_seqs = [(fasta.id, str(fasta.seq)) for fasta in fasta_seqs]
    S = 0 if len(name_seqs[0][1]) <= len(name_seqs[1][1]) else 1
    T = 0 if S == 1 else 1

    seqs = {
        'S': {'name': name_seqs[S][0], 'seq': name_seqs[S][1]},
        'T': {'name': name_seqs[T][0], 'seq': name_seqs[T][1]}
    }
    return seqs


In [11]:
match_score    =  1
mismatch_score = -1
b              = -1
s='AGCTA'
t='AGT'
A,S,T,OPT=filling_matrices(b,s,t,match_score,mismatch_score)
print(A)
print(S)
print(T)
print(OPT)
print(OPT['M'][3][2])
values=[A[4][3],S[4][3],T[4][3]]
print(values.index(max(values)))
new_aln=['','']
aln=back_trace(OPT,'T_gap',4,2,s,t)
print(aln)

[[  0. -inf -inf -inf]
 [-inf   1.  -2. -inf]
 [-inf  -2.   2.  -1.]
 [-inf -inf  -1.   1.]
 [-inf -inf  -4.   2.]
 [-inf -inf -inf  -3.]]
[[  0. -inf -inf -inf]
 [ -1. -inf   0.  -3.]
 [-inf -inf  -1.   1.]
 [-inf -inf  -4.   0.]
 [-inf -inf -inf  -3.]
 [-inf -inf -inf  -6.]]
[[  0.  -1. -inf -inf]
 [-inf -inf -inf -inf]
 [-inf   0.  -1.  -4.]
 [-inf  -3.   1.   0.]
 [-inf -inf  -2.   0.]
 [-inf -inf  -5.   1.]]
{'M': array([[-inf, -inf, -inf, -inf],
       [-inf,   0.,   2.,   0.],
       [-inf,   1.,   0.,   1.],
       [-inf,   0.,   2.,   0.],
       [-inf,   0.,   2.,   2.],
       [-inf,   0.,   0.,   2.]]), 'S_gap': array([[-inf, -inf, -inf, -inf],
       [-inf,   0.,   0.,   0.],
       [-inf,   0.,   2.,   0.],
       [-inf,   0.,   2.,   2.],
       [-inf,   0.,   0.,   2.],
       [-inf,   0.,   0.,   2.]]), 'T_gap': array([[-inf, -inf, -inf, -inf],
       [-inf,   0.,   0.,   0.],
       [-inf,   0.,   1.,   1.],
       [-inf,   0.,   0.,   1.],
       [-inf,   0.,   0.,  

In [27]:
def write_seq_to_file(filename,seq1,seq2,score):
    with open("test.txt", "a") as myfile:
        #print(filename,type(filename))
        myfile.write("score of "+filename+" is "+str(score)+"\n")
        myfile.write(seq1+"\n") 
        myfile.write(seq2+"\n")
        

In [28]:
match_score    =  1
mismatch_score = -1
b              = -1
filenames = [
        'hw1_short.fa.txt',
        'hw1_medium.fa.txt',
        'hw1_long.fa.txt',
        ]
for filename in filenames:
    seqs=load_fasta(filename)
    #print(seqs)
    A,S,T,OPT=filling_matrices(b,seqs['S']['seq'],seqs['T']['seq'],match_score,mismatch_score)
    m=len(seqs['S']['seq'])
    n=len(seqs['T']['seq'])
    values=[A[m][n],S[m][n],T[m][n]]
    table_name=['M','S_gap','T_gap']
    ind=values.index(max(values))
    aln=back_trace(OPT,table_name[ind],m-1,n-1,seqs['S']['seq'],seqs['T']['seq'])
    print(aln,'score is',max(values))
    write_seq_to_file(filename,aln[0],aln[1],max(values))

CAAGGAACTAGTTGAGGTCTGTCTACCCTGTGAGATTCTAAAAGACGTATCGTAATTACTGTAAAACATGAAGAAGTT-G-CGCTTCTCGTCTATTTAGGTA CAAGAAACTAGTCAAGGTTTGTCTAGCCTGTGAGATTCTAAAAGACGTATCGTACTTCCTGTAAAACATGAAGAACTTAAAC-CTGC-CGTCTATTTAGGTA -1 -1
('ATGGATTTATCTGCTCTTCGC-G-TTGAAGAAGTACAAAATGTCATTAATGCTATGCAGAAAATCTTAGAGTGTCCCATCTGTCTGGAGTTGATCAAGGAAC', 'ATGGATTTATCTGC-CGTC-CAAATTCAAGAAGTACAAAATGTCCTTCATGCTATGCAGAAAATCTTAGAGTGTCCGATCTGTTTGGAACTGATCAAAGAAC') score is 74.0
CA-G-GA-GTCTCAAGAGTG-TCAAGGTTCCAATCTCTCAACCTGTG-ACTCTGACCAAAGGACGTTCCTTCC-TAAAAGCCCAAG-TGAGACATCTTCAGAAAACCGTGCCAAAGACATCGGGTATGAAACCTACTATCTTTGAAGTAGAAAATCTACAAGTCCTCTCAATAAA-AGGAAAAAACGTTTTAATATCGACAAACGTAT-GAGGTTTGGACACAGTTCGACTTTTCGT-GTTTACTAAAAGTTATCGAGAAGTTGTTCAACTGATTTAGAGCATGAAAGAACATCCGAGGAAAACCAATATAGTAAGAATGTATTTCCTGTGACACTTCCGGGAAAGAAGACCAACTCTTCAAAGTCGTACGTTTTAAACGTTTTATACACCAGTGTGAAACACCTCTGTCCAAGGAACTAGTTGAGGTCTGTCTACCCTGTGAGATTCTAAAAGACGTATCGTAATTACTGTAAAACATGAAGAAGTT-G-CGCTTCTCGTCTATTTAGGTA AAAAAGAAGTGACTAGAGTGCT-AAGGTTCCAATCTGTCGACCTGTGG